In [1]:
import numpy as np
import pandas as pd

In [2]:
def rle2mask(mask_rle, shape=(1600,256)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (width,height) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T


#https://www.kaggle.com/bguberfain/memory-aware-rle-encoding
def rle_encode_less_memory(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    This simplified method requires first and last pixel to be zero
    '''
    pixels = img.T.flatten()
    
    # This simplified method requires first and last pixel to be zero
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)

def image_size_dict(img_id, x, y):
    image_id = [thing[:-5] for thing in img_id]
    x_y = [(x[i], y[i]) for i in range(0, len(x))]    
    return dict(zip(image_id, x_y))


def global_shift_mask(maskpred1, y_shift, x_shift):
    """
    applies a global shift to a mask by padding one side and cropping from the other
    """
    if y_shift <0 and x_shift >=0:
        maskpred2 = np.pad(maskpred1, [(0,abs(y_shift)), (abs(x_shift), 0)], mode='constant', constant_values=0)
        maskpred3 = maskpred2[abs(y_shift):, :maskpred1.shape[1]]
    elif y_shift >=0 and x_shift <0:
        maskpred2 = np.pad(maskpred1, [(abs(y_shift),0), (0, abs(x_shift))], mode='constant', constant_values=0)
        maskpred3 = maskpred2[:maskpred1.shape[0], abs(x_shift):]
    elif y_shift >=0 and x_shift >=0:
        maskpred2 = np.pad(maskpred1, [(abs(y_shift),0), (abs(x_shift), 0)], mode='constant', constant_values=0)
        maskpred3 = maskpred2[:maskpred1.shape[0], :maskpred1.shape[1]]
    elif y_shift < 0 and x_shift < 0:
        maskpred2 = np.pad(maskpred1, [(0, abs(y_shift)), (0, abs(x_shift))], mode='constant', constant_values=0)
        maskpred3 = maskpred2[abs(y_shift):, abs(x_shift):]
    return maskpred3

In [3]:
dfpred = pd.read_csv('../input/best-public-hubmap/submission_public_TPU.csv')
TARGET_ID = 'afa5e8098'
y_shift = -40
x_shift = -24

In [4]:
#get image sizes 

dfinfo = pd.read_csv('../input/hubmap-kidney-segmentation/HuBMAP-20-dataset_information.csv')

size_dict = image_size_dict(dfinfo.image_file, dfinfo.width_pixels, dfinfo.height_pixels)  #dict which contains image sizes mapped to id's
mask_shape = size_dict.get(TARGET_ID)

taridx = dfpred[dfpred['id']==TARGET_ID].index.values[0]  #row of TARGET_ID in dfpred

In [5]:
maskpred = rle2mask(dfpred.iloc[taridx]['predicted'], mask_shape)

maskpred1 = maskpred.copy()
maskpred1[maskpred1>0]=1

In [6]:
mask_shifted = global_shift_mask(maskpred1, y_shift, x_shift)  #apply specified shift to mask

In [7]:
newrle = rle_encode_less_memory(mask_shifted)  #rle encode shifted mask

dfpred.at[taridx, 'predicted'] = newrle

dfsample = pd.read_csv('../input/hubmap-kidney-segmentation/sample_submission.csv')

mydict = dict(zip(dfpred['id'], dfpred['predicted']))

dfsample['predicted'] = dfsample['id'].map(mydict).fillna(dfsample['predicted'])

dfsample = dfsample.replace(np.nan, '', regex=True)

dfsample.to_csv('submission.csv',index=False)

In [8]:
dfsample.head()

,id,predicted
0,2ec3f1bb9,
1,3589adb90,
2,d488c759a,
3,aa05346ff,
4,57512b7f1,
